In [1]:
import json
import pandas as pd
import pickle

In [2]:
with open('runs/models/summary.pk', 'rb') as f:
    summary = pickle.load(f)

In [3]:
summary['model_11']

{'rewards': {'random': 0.5623922460053068,
  'baseline': 0.5748980923835975,
  'rl': 0.6830969052665726},
 'episode': 5900,
 'target': {'train': 0.7614248866698287, 'val': 0.7598680433208965}}

In [4]:
from framework import *
from joblib import Parallel, delayed
import torch

data_dir = './job_data'
features_path = f'{data_dir}/features.csv'
clf_path =  f'{data_dir}/classifiers.pkl'
settings_path = f'{data_dir}/settings8.json'
reward_path = f'{data_dir}/exhaustive.csv'
splits_path = f'{data_dir}/splits.pk'
model_path = 'runs/models/model_11.bin'

setup = SetUp(features_path, clf_path, splits_path, settings_path, reward_path)
rl_args = setup.rl_kwargs()
rl_args['model_path'] = model_path
rl_eval = RLEval(**rl_args)

100%|██████████| 100/100 [00:04<00:00, 24.66it/s]


In [30]:
cases_rl, rewards_rl = rl_eval.get_rewards(set_key='test', strategy_key='rl')
cases_baseline, rewards_baseline = rl_eval.get_rewards(set_key='test', strategy_key='baseline')
cases_random, rewards_random = rl_eval.get_rewards(set_key='test', strategy_key='random')

There is no experience in memory similar to                    case with id:3694880517013152256 and settings:(10, 1, 0)
There is no experience in memory similar to                    case with id:3694880517013152256 and settings:(10, 1, 0)
There is no experience in memory similar to                    case with id:3694880517013152256 and settings:(10, 1, 0)


In [66]:
from trees import CaseNode

def cases_strategy(setup, rl_eval, n_spec, n_color, n_obs):
    
    assert n_obs in [0,1]
    
    case_settings = setup.env.case_settings.copy()
    cases_photo = []
    for i, ftrs in rl_eval.test_ftrs.iterrows():
        case_settings['n_spec'] = n_spec
        case_settings['n_color'] = n_color
        case_settings['n_obs'] = ftrs.lengths_gaia*n_obs
        case_settings['all_features'] = ftrs
        case_i = CaseNode(**case_settings)
        cases_photo.append(case_i)
    
    return cases_photo

cases_photo = cases_strategy(setup, rl_eval, n_spec=0, n_color=0, n_obs=1)
cases_photo_color = cases_strategy(setup, rl_eval, n_spec=0, n_color=1, n_obs=1)
cases_photo_spec = cases_strategy(setup, rl_eval, n_spec=1, n_color=0, n_obs=1)
cases_full = cases_strategy(setup, rl_eval, n_spec=1, n_color=1, n_obs=1)

In [95]:
from sklearn.metrics import classification_report

y_true_rl = [c.label for c in cases_rl]
y_hat_rl = [c.prediction for c in cases_rl]
report_rl = classification_report(y_true_rl, y_hat_rl, output_dict=True)

y_true_baseline = [c.label for c in cases_baseline]
y_hat_baseline = [c.prediction for c in cases_baseline]
report_baseline = classification_report(y_true_baseline, y_hat_baseline, output_dict=True)

y_true_random = [c.label for c in cases_random]
y_hat_random = [c.prediction for c in cases_random]
report_random = classification_report(y_true_random, y_hat_random, output_dict=True)

y_true_photo = [c.label for c in cases_photo]
y_hat_photo = [c.prediction for c in cases_photo]
report_photo = classification_report(y_true_photo, y_hat_photo, output_dict=True)

y_true_photo_color = [c.label for c in cases_photo_color]
y_hat_photo_color = [c.prediction for c in cases_photo_color]
report_photo_color = classification_report(y_true_photo_color, y_hat_photo_color, output_dict=True)

y_true_photo_spec = [c.label for c in cases_photo_spec]
y_hat_photo_spec = [c.prediction for c in cases_photo_spec]
report_photo_spec = classification_report(y_true_photo_spec, y_hat_photo_spec, output_dict=True)

y_true_full = [c.label for c in cases_full]
y_hat_full = [c.prediction for c in cases_full]
report_full = classification_report(y_true_full, y_hat_full, output_dict=True)

In [97]:
report_rl['accuracy']

0.9316037735849056

In [109]:
n_obs_rl = np.mean([c.n_obs for c in cases_rl])
n_obs_baseline = np.mean([c.n_obs for c in cases_baseline])
n_obs_random = np.mean([c.n_obs for c in cases_random])
n_obs_photo = np.mean([c.n_obs for c in cases_photo])
n_obs_photo_color = np.mean([c.n_obs for c in cases_photo_color])
n_obs_photo_spec = np.mean([c.n_obs for c in cases_photo_spec])
n_obs_full = np.mean([c.n_obs for c in cases_full])

In [123]:
n_spec_rl = np.mean([c.n_spec for c in cases_rl])
n_spec_baseline = np.mean([c.n_spec for c in cases_baseline])
n_spec_random = np.mean([c.n_spec for c in cases_random])
n_spec_photo = np.mean([c.n_spec for c in cases_photo])
n_spec_photo_color = np.mean([c.n_spec for c in cases_photo_color])
n_spec_photo_spec = np.mean([c.n_spec for c in cases_photo_spec])
n_spec_full = np.mean([c.n_spec for c in cases_full])

In [132]:
n_color_rl = np.mean([c.n_color for c in cases_rl])
n_color_baseline = np.mean([c.n_color for c in cases_baseline])
n_color_random = np.mean([c.n_color for c in cases_random])
n_color_photo = np.mean([c.n_color for c in cases_photo])
n_color_photo_color = np.mean([c.n_color for c in cases_photo_color])
n_color_photo_color = np.mean([c.n_color for c in cases_photo_color])
n_color_full = np.mean([c.n_color for c in cases_full])

In [88]:
test_ids = [c.survey_id for c in cases_rl]

exhaustive_path = '../costs/exhaustive.csv'
exhaustive_df = pd.read_csv(exhaustive_path)
exhaustive_test = exhaustive_df.set_index('gaia_id').loc[test_ids]

In [134]:
acc_exhaustive = np.mean(exhaustive_test.label==exhaustive_test.prediction)
n_obs_exhaustive = np.mean(exhaustive_test.photo)
n_spec_exhaustive = np.mean(exhaustive_test.spectrum)
n_color_exhaustive = np.mean(exhaustive_test.color) 

In [127]:
exhaustive_test

,color,cost,label,photo,prediction,probability,reward,spectrum,strategy
gaia_id,,,,,,,,,
633961568304132352,0,5,RRAB,5,RRAB,1.000000,0.875000,0,brute_force
1037688635863106048,0,8,RRAB,8,RRAB,1.000000,0.800000,0,brute_force
1742456480958663040,1,6,RRC,5,RRC,0.724138,0.574138,0,brute_force
1312081074064627072,0,5,RRAB,5,RRAB,1.000000,0.875000,0,brute_force
1489271732271232768,1,8,RRC,7,RRC,0.931034,0.731034,0,brute_force
3867023798373308544,1,11,RRAB,5,RRAB,0.864865,0.589865,1,brute_force
1541258536115214464,0,9,RRAB,9,RRAB,1.000000,0.775000,0,brute_force
3686823806904763136,1,6,RRAB,5,RRAB,1.000000,0.850000,0,brute_force
1657334352318594816,0,5,RRAB,5,RRAB,1.000000,0.875000,0,brute_force


In [135]:
strategies = [
    'rl',
    'baseline',
    'random',
    'photo',
    'photo_color',
    'photo_spec',
    'full',
    'exhaustive'
]

accuracies = [
    report_rl['accuracy'],
    report_baseline['accuracy'],
    report_random['accuracy'],
    report_photo['accuracy'],
    report_photo_color['accuracy'],
    report_photo_spec['accuracy'],
    report_full['accuracy'],
    acc_exhaustive
]

n_spec = [
    n_spec_rl,
    n_spec_baseline,
    n_spec_random,
    n_spec_photo,
    n_spec_photo_color,
    n_spec_photo_spec,
    n_spec_full,
    n_spec_exhaustive
]

n_color = [
    n_color_rl,
    n_color_baseline,
    n_color_random,
    n_color_photo,
    n_color_photo_color,
    n_color_photo_color,
    n_color_full,
    n_color_exhaustive
]

results = {'strategies':strategies, 'accuracy': accuracies, 'n_photo': n_obs, 'n_spec':n_spec,
           'n_color': n_color}

In [136]:
pd.DataFrame(results)

,strategies,accuracy,n_photo,n_spec,n_color
0,rl,0.931604,6.034198,0.001179,0.548349
1,baseline,0.883255,5.846698,0.520047,0.482311
2,random,0.895047,6.619104,0.489387,0.500000
3,photo,0.902123,13.319575,0.000000,0.000000
4,photo_color,0.933962,13.319575,0.000000,1.000000
5,photo_spec,0.922170,13.319575,1.000000,1.000000
6,full,0.948113,13.319575,1.000000,1.000000
7,exhaustive,0.987028,6.121462,0.033019,0.510613


In [104]:
aux = cases_rl[0]

In [105]:
aux.n_obs

5